In [10]:
import torch
from transformers import BertTokenizer, BertModel
import tqdm

In [11]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
model.eval()  # Set the model to evaluation mode if not training

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [3]:
texts = [
    "Yes, we will keep all the materials after the course finishes.",
    "You can follow the course at your own pace after it finishes"
]
encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

In [4]:
with torch.no_grad():  # Disable gradient calculation for inference
    outputs = model(**encoded_input)
    hidden_states = outputs.last_hidden_state

In [5]:
sentence_embeddings = hidden_states.mean(dim=1)
sentence_embeddings.shape

torch.Size([2, 768])

In [6]:
sentence_embeddings_cpu = sentence_embeddings.cpu()

In [7]:
X_emb = sentence_embeddings_cpu.numpy()

In [8]:
def make_batches(seq, n):
    result = []
    for i in range(0, len(seq), n):
        batch = seq[i:i+n]
        result.append(batch)
    return result

In [9]:
import requests
import pandas as pd
import numpy as np

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs = requests.get(docs_url)
raw_docs = docs.json()

documents = []
for course in raw_docs:
    course_name = course['course']
    
    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

data = pd.DataFrame(documents, columns=['text','section','question','course'])

df = data[data['course'] == 'data-engineering-zoomcamp']

In [15]:
texts = df['text'].tolist()
text_batches = make_batches(texts, 8)

all_embeddings = []

for batch in text_batches:
    encoded_input = tokenizer(batch, padding=True, truncation=True, return_tensors='pt')

    with torch.no_grad():
        outputs = model(**encoded_input)
        hidden_states = outputs.last_hidden_state
        
        batch_embeddings = hidden_states.mean(dim=1)
        batch_embeddings_np = batch_embeddings.cpu().numpy()
        all_embeddings.append(batch_embeddings_np)

final_embeddings = np.vstack(all_embeddings)

In [16]:
final_embeddings

array([[-0.00456303, -0.11667512,  0.6274718 , ..., -0.03659191,
         0.10031679,  0.0292713 ],
       [-0.1423361 , -0.1985392 ,  0.28455415, ..., -0.01139053,
        -0.1539977 ,  0.09535079],
       [ 0.19672246, -0.08461305,  0.28200513, ...,  0.11395867,
        -0.06448027, -0.0128261 ],
       ...,
       [ 0.01786945, -0.29183316,  0.2481716 , ..., -0.15940122,
        -0.21750253,  0.00608711],
       [-0.09272221, -0.00315099,  0.4262845 , ..., -0.13433972,
        -0.11954762,  0.29828972],
       [-0.18737537,  0.06122091,  0.5668274 , ..., -0.21628517,
        -0.23186603,  0.14869094]], dtype=float32)